### Transfer learning with ResNet18
Source: https://www.kaggle.com/pintu161/transfer-learning-in-pytorch-using-resnet18

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
from collections import OrderedDict
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [ ]:
# download the pretrained model
import torchvision.models as models
model = models.resnet18(pretrained = True)
print(model)

#switch device to gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## Load Data

In [ ]:
# Unzip files
import zipfile
extract_files = ['train', 'test1']
for file in extract_files:
    with zipfile.ZipFile("../input/"+file+".zip", "r") as z:
        z.extractall(".")

In [ ]:
# Load the list of files
train_dir = 'train'
test_dir = 'test1'
train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)

In [ ]:
len(train_files)

In [ ]:
len(test_files)

In [ ]:
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from PIL import Image
import torchvision

class CatDogDataset(Dataset):
    def __init__(self, file_list, dir, mode='train', transform = None):
        self.file_list = file_list
        self.dir = dir
        self.mode= mode
        self.transform = transform
        if self.mode == 'train':
            if 'dog' in self.file_list[0]:
                self.label = 1
            else:
                self.label = 0
            
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.dir, self.file_list[idx]))
        if self.transform:
            img = self.transform(img)
        if self.mode == 'train':
            img = img.numpy()
            return img.astype('float32'), self.label
        else:
            img = img.numpy()
            return img.astype('float32'), self.file_list[idx]
        
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ColorJitter(),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(128),
    transforms.ToTensor()
])

cat_files = [tf for tf in train_files if 'cat' in tf]
dog_files = [tf for tf in train_files if 'dog' in tf]

cats = CatDogDataset(cat_files, train_dir, transform = data_transform)
dogs = CatDogDataset(dog_files, train_dir, transform = data_transform)

catdogs = ConcatDataset([cats, dogs])

In [ ]:
dataloader = DataLoader(catdogs, batch_size = 64, shuffle=True, num_workers=4)

In [ ]:
samples, labels = iter(dataloader).next()
plt.figure(figsize=(16,24))
grid_imgs = torchvision.utils.make_grid(samples[:24])
np_grid_imgs = grid_imgs.numpy()
# in tensor, image is (batch, width, height), so you have to transpose it to (width, height, batch) in numpy to show it.
plt.imshow(np.transpose(np_grid_imgs, (1,2,0)))

### Simple CNN model

In [ ]:
class CnnNet(nn.Module):
    def __init__(self):
        super(CnnNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 7, 2)
        self.conv2 = nn.Conv2d(64, 128, 3, 2)
        self.dropout1 = nn.Dropout2d(0.1)
        self.dropout2 = nn.Dropout2d(0.1)
        self.fc1 = nn.Linear(4608, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 3, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 3, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
net = CnnNet()
net = net.to(device)

## Training Only the Classifier

In [ ]:
# Freeze the parameters 
for param in model.parameters():
    param.requires_grad = False 
    
# Classifier architecture to put on top of resnet18
fc = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(512,100)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(100,2)),
    ('output', nn.LogSoftmax(dim=1))
]))

model.fc = fc

# shifting model to gpu
model.to(device)
model

In [ ]:
device

In [ ]:
from tqdm import tqdm_notebook as tqdm
# function to train the model
def train(model, trainloader, criterion, optimizer, epochs = 5, log_interval=100):
    train_loss = []
    all_loss = []
    for epoch in range(epochs):
        running_loss = 0
        tq = tqdm(trainloader)
        for batch_idx, (images, labels) in enumerate(tq):
            inputs, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            img = model(inputs)
            
            loss = criterion(img, labels)
            all_loss.append(loss.item())
            running_loss += loss
            loss.backward()
            optimizer.step()
            
            if batch_idx % log_interval == 0:
                tq.set_description(f'Epoch {epoch}: {(running_loss / (batch_idx+1)).item():.3f}')
            
        train_loss.append(running_loss/len(trainloader))
        
    return train_loss, all_loss

In [ ]:
# Train simple CNN
epochs = 1
net.train()
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.NLLLoss()    
train_loss, all_loss = train(net,dataloader,criterion, optimizer, epochs, log_interval=10) 

In [ ]:
# Train pre-trained ResNet
epochs = 1
model.train()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
criterion = nn.NLLLoss()    
ft_train_loss, ft_all_loss = train(model,dataloader,criterion, optimizer, epochs, log_interval=10) 

In [ ]:
391 * 64

In [ ]:
# Plot the 2 loss curve
plt.figure(figsize=(20, 8))
plt.plot(all_loss)
plt.plot(ft_all_loss)
plt.legend(['Simple CNN from scratch', 'Finetuned ResNet'])
plt.ylabel('loss')
plt.xlabel('batch number')

In [ ]:
# Save the model
# filename_pth = 'ckpt_resnet18_catdog.pth'
# torch.save(model.state_dict(), filename_pth)


# Transform the test dataset
test_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

testset = CatDogDataset(test_files, test_dir, mode='test', transform = test_transform)
testloader = DataLoader(testset, batch_size = 64, shuffle=False, num_workers=4)

## Prediction on Testset

In [ ]:
model.eval()
fn_list = []
pred_list = []
for x, fn in testloader:
    with torch.no_grad():
        x = x.to(device)
        output = model(x)
        pred = torch.argmax(output, dim=1)
        fn_list += [n[:-4] for n in fn]
        pred_list += [p.item() for p in pred]

submission = pd.DataFrame({"id":fn_list, "label":pred_list})
# submission.to_csv('preds_resnet18.csv', index=False)

In [ ]:
samples, _ = iter(testloader).next()
samples = samples.to(device)
fig = plt.figure(figsize=(24, 16))
fig.tight_layout()
output = model(samples[:24])
pred = torch.argmax(output, dim=1)
pred = [p.item() for p in pred]
ad = {0:'cat', 1:'dog'}
for num, sample in enumerate(samples[:24]):
    plt.subplot(4,6,num+1)
    plt.title(ad[pred[num]])
    plt.axis('off')
    sample = sample.cpu().numpy()
    plt.imshow(np.transpose(sample, (1,2,0)))

In [ ]:
samples, _ = iter(testloader).next()
samples = samples.to(device)
fig = plt.figure(figsize=(24, 16))
fig.tight_layout()
output = net(samples[:24])
pred = torch.argmax(output, dim=1)
pred = [p.item() for p in pred]
ad = {0:'cat', 1:'dog'}
for num, sample in enumerate(samples[:24]):
    plt.subplot(4,6,num+1)
    plt.title(ad[pred[num]])
    plt.axis('off')
    sample = sample.cpu().numpy()
    plt.imshow(np.transpose(sample, (1,2,0)))

References:<br>
[densenet](https://www.kaggle.com/jaeboklee/pytorch-cat-vs-dog) <br>
[Udacity Transfer Learning](https://classroom.udacity.com/courses/ud188/lessons/c5706f76-0e30-4b48-b74e-c19fafc33a75/concepts/c33dec4c-ff16-465f-88e9-e95365e7b522)